Transformación y manipulación de datos (Parte II)
Pregunta 1:
Cargar los datos desde el excel US_Regional_Sales_Data.xlsx, para esto utilizamos primeramente .sheet_names para conocer los nombres de las hojas que utilizaremos para nuestro DF.
Luego de esto almacenaremos los datos en una variable para luego transformarla en un DF, aplicando esto a cada una de las hojas del excel.

In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Revisar los nombres de las hojas de xlsx
pd.ExcelFile('US_Regional_Sales_Data.xlsx').sheet_names

['Sales Orders Sheet',
 'Customers Sheet',
 'Store Locations Sheet',
 'Products Sheet',
 'Sales Team Sheet']

In [3]:
data_sales = pd.read_excel('US_Regional_Sales_Data.xlsx', sheet_name='Sales Orders Sheet')
df_sales = pd.DataFrame(data_sales)
df_sales.head(3)

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.075,1963.1,1001.181
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.075,3939.6,3348.660
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.050,1775.5,781.220


In [4]:
data_customers = pd.read_excel("US_Regional_Sales_Data.xlsx", sheet_name='Customers Sheet')
df_customers = pd.DataFrame(data_customers)
df_customers.head(2)

,_CustomerID,Customer Names
0,1,Avon Corp
1,2,WakeFern


In [5]:
data_stores_lz = pd.read_excel("US_Regional_Sales_Data.xlsx", sheet_name='Store Locations Sheet')
df_stores_lz = pd.DataFrame(data_stores_lz)
df_stores_lz.head(2)

,_StoreID,City Name,County,StateCode,State,Type,Latitude,Longitude,AreaCode,Population,Household Income,Median Income,Land Area,Water Area,Time Zone
0,1,Birmingham,Shelby County/Jefferson County,AL,Alabama,City,33.52744,-86.79905,205,212461,89972,31061,378353942,6591013,America/Chicago
1,2,Huntsville,Limestone County/Madison County,AL,Alabama,City,34.69901,-86.67298,256,190582,78554,48775,552604579,3452021,America/Chicago


In [6]:
data_products = pd.read_excel("US_Regional_Sales_Data.xlsx", sheet_name='Products Sheet')
df_products = pd.DataFrame(data_products)
df_products.head(2)

,_ProductID,Product Name
0,1,Cookware
1,2,Photo Frames


In [7]:
data_sales_team = pd.read_excel("US_Regional_Sales_Data.xlsx", sheet_name='Sales Team Sheet')
df_team = pd.DataFrame(data_sales_team)
df_team.head(2)

,_SalesTeamID,Sales Team,Region
0,1,Adam Hernandez,Northeast
1,2,Keith Griffin,Northeast


In [8]:
#Crear merge de DF para generar el DF base al que agregaremos las columnas calculadas
df_base = df_sales.merge(
    df_customers, 
    on='_CustomerID',
    how='outer',
    validate='many_to_one'   
).merge(
    df_stores_lz,
    on='_StoreID',
    how='outer',
    validate='many_to_one'
).merge(
    df_products,
    on='_ProductID',
    how='outer',
    validate='many_to_one'
).merge(
    df_team,
    on='_SalesTeamID',
    how='outer',
    validate='many_to_one'
)
df_base.head(3)

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,AreaCode,Population,Household Income,Median Income,Land Area,Water Area,Time Zone,Product Name,Sales Team,Region
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,631,213776,68789,80327,135481314,160302131,America/New York,Dining Furniture,Joshua Bennett,Northeast
1,SO - 0002491,In-Store,WARE-NMK1003,2018-10-27,2019-03-06,2019-03-25,2019-03-30,USD,6,11,...,478,153515,57025,36568,645603627,14488557,America/New York,Dining Furniture,Joshua Bennett,Northeast
2,SO - 0001458,In-Store,WARE-NMK1003,2018-07-19,2018-11-09,2018-11-11,2018-11-17,USD,6,12,...,719,456568,174441,54527,506667557,947196,America/Denver,Dining Furniture,Joshua Bennett,Northeast


In [9]:
#Transformar las columnas que utilizaremos de fechas a formato de fecha
df_base['OrderDate']= pd.to_datetime(df_base['OrderDate'])
df_base['ProcuredDate']= pd.to_datetime(df_base['ProcuredDate'])
df_base['ShipDate']= pd.to_datetime(df_base['ShipDate'])
df_base['DeliveryDate']= pd.to_datetime(df_base['DeliveryDate'])

In [10]:
#Calcular días para las columnas calculadas
df_base['ProcurementDays'] = df_base['OrderDate']-df_base['ProcuredDate']
df_base['ShippingDays']= df_base['ShipDate']-df_base['OrderDate']
df_base['DeliveryDays']= df_base['DeliveryDate']-df_base['ShipDate']

In [11]:
#Convertir los CostumerDays y transformarlos a numeric para aplicar formula que procese los segundos y devuelva la cantidad de días
df_base['CustomerDays']= df_base['ShippingDays']+df_base['DeliveryDays']
df_base['CustomerDays']=df_base['CustomerDays']= pd.to_numeric(df_base['CustomerDays'])/10**9/(86400)
df_base['CustomerDays']

0       19.0
1       24.0
2        8.0
3        9.0
4       12.0
        ... 
7986    32.0
7987    38.0
7988    27.0
7989    13.0
7990    29.0
Name: CustomerDays, Length: 7991, dtype: float64

In [12]:
#Con método .cut() etiquetamos y discretizamos los datos para nuestra tabla pivote
intervals = [0,15,30,45,60,75,90]
labels_days = ['0 to 15 days', '15 to 30 dyas', '30 to 45 days', '45 to 60 days', '60 to 75 days', '75 to 90 days']
df_base['CustomerDaysInterval'] = pd.cut(df_base['CustomerDays'], bins= intervals, labels=labels_days)
df_base['CustomerDaysInterval']

0       15 to 30 dyas
1       15 to 30 dyas
2        0 to 15 days
3        0 to 15 days
4        0 to 15 days
            ...      
7986    30 to 45 days
7987    30 to 45 days
7988    15 to 30 dyas
7989     0 to 15 days
7990    15 to 30 dyas
Name: CustomerDaysInterval, Length: 7991, dtype: category
Categories (6, object): ['0 to 15 days' < '15 to 30 dyas' < '30 to 45 days' < '45 to 60 days' < '60 to 75 days' < '75 to 90 days']

In [13]:
#Tabla pivote para tener los intervalos por equipo de ventas
df_pivot1 = pd.pivot_table(
                           data=df_base
                          ,index='Sales Team'
                          ,columns='CustomerDaysInterval'
                          ,values='Order Quantity'
                          ,aggfunc='count'                          
)
df_pivot1

CustomerDaysInterval,0 to 15 days,15 to 30 dyas,30 to 45 days,45 to 60 days,60 to 75 days,75 to 90 days
Sales Team,,,,,,
Adam Hernandez,81,179,42,0,0,0
Anthony Berry,76,181,41,0,0,0
Anthony Torres,81,162,45,0,0,0
Carl Nguyen,96,178,40,0,0,0
Carlos Miller,65,148,34,0,0,0
Chris Armstrong,92,156,44,0,0,0
Donald Reynolds,97,163,36,0,0,0
Douglas Tucker,87,145,45,0,0,0
Frank Brown,85,150,40,0,0,0


In [14]:
#Agregar columna GrossMargin a partir del calculo de varias columnas de nuestro DF
df_base['GrossMargin'] = df_base['Order Quantity']*(df_base['Unit Price']*(1-df_base['Discount Applied'])-df_base['Order Quantity'])
pd.to_numeric(df_base['GrossMargin'])
df_base['GrossMargin']=df_base['GrossMargin'].astype('Float64')

In [15]:
#Con método .cut() discretizamos nuevamente los datos y luego de esto lo cambiaremos a numerico para procesarlo en una tabla pivote. Adicionalmente para obtener el resultado del siguiente ejercicio
#utilizamos los porcentajes como decimales para obtener el monto de la comisión
intervals2 = [0,100,1000,10000,100000]
labels3 = [0.5,0.10,0.15,0.20]
df_base['ComissionsPercentage'] = pd.cut(df_base['GrossMargin'], bins=intervals2,labels=labels3)
df_base['ComissionsPercentage']=pd.to_numeric(df_base['ComissionsPercentage'])
df_base['ComissionsPercentage'].head(3)

0    0.15
1    0.15
2    0.15
Name: ComissionsPercentage, dtype: float64

In [16]:
#CommissionAmount
df_base['CommissionAmount'] = df_base['GrossMargin']*df_base['ComissionsPercentage']
df_base['CommissionAmount']

0       1358.150625
1         931.47075
2           840.228
3        1356.76875
4            637.83
           ...     
7986       573.9675
7987         213.51
7988       1316.979
7989      755.27625
7990        640.413
Name: CommissionAmount, Length: 7991, dtype: Float64

In [17]:
#NetMargin
df_base['NetMargin'] = df_base['GrossMargin']-df_base['CommissionAmount']
df_base['NetMargin'].head(3)

0    7696.186875
1     5278.33425
2       4761.292
Name: NetMargin, dtype: Float64

In [96]:
df_pivot_FINAL = pd.pivot_table(
                                data=df_base
                               ,index='Sales Team'
                               ,values=['CommissionAmount','GrossMargin', 'NetMargin']
                               ,aggfunc='sum'
)
df_pivot_FINAL

,CommissionAmount,GrossMargin,NetMargin
Sales Team,,,
Adam Hernandez,525849.958,2817849.3225,2291999.3645
Anthony Berry,476866.87375,2623994.845,2147127.97125
Anthony Torres,476992.031875,2583988.8425,2106996.810625
Carl Nguyen,510465.576375,2778515.6225,2268050.046125
Carlos Miller,399970.859,2189690.735,1789719.876
Chris Armstrong,464746.505625,2528673.1125,2063926.606875
Donald Reynolds,557458.099875,2972260.4275,2414802.327625
Douglas Tucker,475333.85,2562702.535,2087368.685
Frank Brown,439133.733125,2400957.47,1961823.736875
